# PyramidNet + ShakeDrop for CIFAR-100

Implementation of state-of-the-art PyramidNet with ShakeDrop regularization achieving ~89.3% top-1 accuracy on CIFAR-100.

<a href="https://colab.research.google.com/github/theboredman/CSE468/blob/main/Quiz_1/CNN/PyramidNet_ShakeDrop_CIFAR100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup and Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR100

import numpy as np
import matplotlib.pyplot as plt
import random
import math
from tqdm import tqdm
import time
import os

# Set device and reproducibility
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Set seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

Using device: cuda


## AutoAugment for CIFAR-100

Implementation of AutoAugment policy specifically optimized for CIFAR-100

In [2]:
class AutoAugmentCIFAR100:
    """AutoAugment policy for CIFAR-100 from the paper"""

    def __init__(self):
        self.policies = [
            # Policy 1
            [("invert", 0.1, 7), ("contrast", 0.2, 6)],
            # Policy 2
            [("rotate", 0.7, 2), ("translateX", 0.3, 9)],
            # Policy 3
            [("shearY", 0.8, 8), ("translateY", 0.7, 9)],
            # Policy 4
            [("posterize", 0.5, 7), ("rotate", 0.9, 3)],
            # Policy 5
            [("solarize", 0.5, 5), ("autocontrast", 0.9, 3)],
            # Policy 6
            [("equalize", 0.8, 8), ("invert", 0.1, 3)],
            # Policy 7
            [("translateY", 0.7, 9), ("autocontrast", 0.9, 1)],
            # Policy 8
            [("solarize", 0.3, 5), ("equalize", 0.4, 4)],
            # Policy 9
            [("solarize", 0.6, 5), ("autocontrast", 0.6, 2)],
            # Policy 10
            [("contrast", 0.6, 7), ("sharpness", 0.6, 5)],
            # Policy 11
            [("brightness", 0.3, 7), ("autocontrast", 0.4, 4)],
            # Policy 12
            [("equalize", 0.6, 4), ("equalize", 0.5, 9)],
            # Policy 13
            [("rotate", 0.9, 8), ("equalize", 0.6, 2)],
            # Policy 14
            [("color", 0.9, 9), ("equalize", 0.6, 6)],
            # Policy 15
            [("autocontrast", 0.8, 4), ("solarize", 0.2, 8)],
            # Policy 16
            [("brightness", 0.1, 3), ("color", 0.7, 0)],
            # Policy 17
            [("solarize", 0.4, 5), ("autocontrast", 0.9, 3)],
            # Policy 18
            [("translateY", 0.9, 9), ("translateY", 0.7, 9)],
            # Policy 19
            [("autocontrast", 0.9, 2), ("solarize", 0.8, 3)],
            # Policy 20
            [("equalize", 0.8, 8), ("invert", 0.1, 3)],
            # Policy 21
            [("translateY", 0.7, 9), ("autocontrast", 0.9, 1)],
            # Policy 22
            [("solarize", 0.3, 5), ("equalize", 0.4, 4)],
            # Policy 23
            [("solarize", 0.6, 5), ("autocontrast", 0.6, 2)],
            # Policy 24
            [("contrast", 0.6, 7), ("sharpness", 0.6, 5)],
            # Policy 25
            [("brightness", 0.3, 7), ("autocontrast", 0.4, 4)]
        ]

    def __call__(self, img):
        policy = random.choice(self.policies)
        for operation, prob, magnitude in policy:
            if random.random() < prob:
                img = self._apply_operation(img, operation, magnitude)
        return img

    def _apply_operation(self, img, operation, magnitude):
        if operation == "rotate":
            angle = magnitude * 30 / 10  # Scale to 0-30 degrees
            return transforms.functional.rotate(img, angle)
        elif operation == "translateX":
            translate = magnitude * 0.45 / 10  # Scale to 0-0.45
            return transforms.functional.affine(img, 0, [translate * img.size[0], 0], 1, 0)
        elif operation == "translateY":
            translate = magnitude * 0.45 / 10
            return transforms.functional.affine(img, 0, [0, translate * img.size[1]], 1, 0)
        elif operation == "shearX":
            shear = magnitude * 0.3 / 10  # Scale to 0-0.3
            return transforms.functional.affine(img, 0, [0, 0], 1, [shear, 0])
        elif operation == "shearY":
            shear = magnitude * 0.3 / 10
            return transforms.functional.affine(img, 0, [0, 0], 1, [0, shear])
        elif operation == "autocontrast":
            return transforms.functional.autocontrast(img)
        elif operation == "invert":
            return transforms.functional.invert(img)
        elif operation == "equalize":
            return transforms.functional.equalize(img)
        elif operation == "solarize":
            threshold = 256 - magnitude * 256 / 10
            return transforms.functional.solarize(img, threshold)
        elif operation == "posterize":
            bits = int(magnitude * 4 / 10) + 4  # 4-8 bits
            return transforms.functional.posterize(img, bits)
        elif operation == "contrast":
            factor = magnitude * 0.9 / 10 + 0.1  # 0.1-1.0
            return transforms.functional.adjust_contrast(img, factor)
        elif operation == "color":
            factor = magnitude * 0.9 / 10 + 0.1
            return transforms.functional.adjust_saturation(img, factor)
        elif operation == "brightness":
            factor = magnitude * 0.9 / 10 + 0.1
            return transforms.functional.adjust_brightness(img, factor)
        elif operation == "sharpness":
            factor = magnitude * 0.9 / 10 + 0.1
            return transforms.functional.adjust_sharpness(img, factor)
        else:
            return img

## Data Loading with Enhanced Augmentation

In [3]:
# CIFAR-100 statistics
CIFAR100_MEAN = [0.5071, 0.4867, 0.4408]
CIFAR100_STD = [0.2675, 0.2565, 0.2761]

# Enhanced training transforms with AutoAugment
train_transform = transforms.Compose([
    # AutoAugment first
    AutoAugmentCIFAR100(),
    # Standard augmentations
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(p=0.5),
    # Cutout augmentation
    transforms.ToTensor(),
    transforms.Normalize(CIFAR100_MEAN, CIFAR100_STD),
    # Random Erasing (similar to Cutout)
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False)
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(CIFAR100_MEAN, CIFAR100_STD)
])

# Load datasets
train_dataset = CIFAR100(root='./data', train=True, download=True, transform=train_transform)
test_dataset = CIFAR100(root='./data', train=False, download=True, transform=test_transform)

print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")

100%|██████████| 169M/169M [00:03<00:00, 43.3MB/s]


Training samples: 50000
Test samples: 10000


## ShakeDrop Regularization

In [4]:
class ShakeDrop(nn.Module):
    """ShakeDrop regularization layer

    Reference: https://arxiv.org/abs/1802.02375
    ShakeDrop regularization for deep residual learning
    """

    def __init__(self, p_drop=0.5, alpha_range=(-1, 1), beta_range=(0, 1)):
        super(ShakeDrop, self).__init__()
        self.p_drop = p_drop
        self.alpha_range = alpha_range
        self.beta_range = beta_range

    def forward(self, x):
        if not self.training:
            return x

        # ShakeDrop gate
        gate = torch.rand(x.size(0), 1, 1, 1, device=x.device)

        # Apply dropout with probability p_drop
        if torch.rand(1).item() < self.p_drop:
            # Random alpha and beta
            alpha = torch.rand(x.size(0), 1, 1, 1, device=x.device) * \
                   (self.alpha_range[1] - self.alpha_range[0]) + self.alpha_range[0]
            beta = torch.rand(x.size(0), 1, 1, 1, device=x.device) * \
                  (self.beta_range[1] - self.beta_range[0]) + self.beta_range[0]

            # Forward: (1 - beta + beta * alpha) * x
            # Backward: (1 - beta + beta * gate) * x
            scale_forward = 1 - beta + beta * alpha
            scale_backward = 1 - beta + beta * gate

            # Use straight-through estimator
            return x * scale_forward + (x * scale_backward - x * scale_forward).detach()
        else:
            return x

## PyramidNet Architecture

In [5]:
class PyramidBottleneck(nn.Module):
    """PyramidNet Bottleneck Block with ShakeDrop"""

    def __init__(self, in_channels, out_channels, stride=1, alpha=0, shake_drop_prob=0.0):
        super(PyramidBottleneck, self).__init__()
        self.stride = stride

        # Calculate bottleneck channels
        bottleneck_channels = out_channels // 4

        # Bottleneck layers
        self.conv1 = nn.Conv2d(in_channels, bottleneck_channels, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(bottleneck_channels)

        self.conv2 = nn.Conv2d(bottleneck_channels, bottleneck_channels, kernel_size=3,
                              stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(bottleneck_channels)

        self.conv3 = nn.Conv2d(bottleneck_channels, out_channels, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels)

        # ShakeDrop regularization
        self.shake_drop = ShakeDrop(p_drop=shake_drop_prob)

        # Shortcut connection
        if stride != 1 or in_channels != out_channels:
            # Use zero-padding for channel expansion (PyramidNet style)
            self.shortcut = nn.Sequential(
                nn.AvgPool2d(kernel_size=stride, stride=stride) if stride > 1 else nn.Identity()
            )
            self.pad_channels = out_channels - in_channels
        else:
            self.shortcut = nn.Identity()
            self.pad_channels = 0

    def forward(self, x):
        residual = self.shortcut(x)

        # Zero-pad channels if needed (PyramidNet characteristic)
        if self.pad_channels > 0:
            batch_size, channels, height, width = residual.shape
            padding = torch.zeros(batch_size, self.pad_channels, height, width,
                                device=residual.device, dtype=residual.dtype)
            residual = torch.cat([residual, padding], dim=1)

        # Main path
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))

        # Apply ShakeDrop
        out = self.shake_drop(out)

        # Add residual
        out = out + residual
        out = F.relu(out)

        return out


class PyramidNet(nn.Module):
    """PyramidNet architecture for CIFAR-100

    Reference: https://arxiv.org/abs/1610.02915
    Deep Pyramidal Residual Networks
    """

    def __init__(self, depth=272, alpha=200, num_classes=100, bottleneck=True):
        super(PyramidNet, self).__init__()

        # Calculate blocks per stage
        if bottleneck:
            assert (depth - 2) % 9 == 0, "depth should be 9n+2 for bottleneck"
            n = (depth - 2) // 9
            block = PyramidBottleneck
        else:
            assert (depth - 2) % 6 == 0, "depth should be 6n+2 for basic block"
            n = (depth - 2) // 6
            # For simplicity, we'll use bottleneck blocks
            block = PyramidBottleneck

        # Initial convolution
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)

        # Calculate channel increments
        self.start_channels = 16
        self.alpha = alpha
        self.total_blocks = 3 * n

        # Build stages
        self.stage1 = self._make_stage(block, 16, n, stride=1, stage=1)
        self.stage2 = self._make_stage(block, self._get_out_channels(n), n, stride=2, stage=2)
        self.stage3 = self._make_stage(block, self._get_out_channels(2*n), n, stride=2, stage=3)

        # Final layers
        self.final_channels = self._get_out_channels(3*n)
        self.bn_final = nn.BatchNorm2d(self.final_channels)
        self.relu_final = nn.ReLU(inplace=True)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(self.final_channels, num_classes)

        # Initialize weights
        self._initialize_weights()

    def _get_out_channels(self, block_idx):
        """Calculate output channels for block_idx using PyramidNet formula"""
        return self.start_channels + int(self.alpha * block_idx / self.total_blocks)

    def _make_stage(self, block, in_channels, num_blocks, stride, stage):
        """Create a stage with num_blocks"""
        layers = []

        for i in range(num_blocks):
            # Calculate block index for ShakeDrop probability
            block_idx = (stage - 1) * num_blocks + i + 1

            # Linear decay of ShakeDrop probability
            shake_drop_prob = 0.5 * block_idx / self.total_blocks

            # Calculate output channels
            out_channels = self._get_out_channels(block_idx)

            # First block in stage may have stride > 1
            block_stride = stride if i == 0 else 1

            layers.append(block(in_channels, out_channels,
                              stride=block_stride,
                              shake_drop_prob=shake_drop_prob))

            in_channels = out_channels

        return nn.Sequential(*layers)

    def _initialize_weights(self):
        """Initialize network weights"""
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                nn.init.constant_(m.bias, 0)

    def forward(self, x):
        # Initial convolution
        x = F.relu(self.bn1(self.conv1(x)))

        # Stages
        x = self.stage1(x)
        x = self.stage2(x)
        x = self.stage3(x)

        # Final layers
        x = self.relu_final(self.bn_final(x))
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x


def pyramidnet272_alpha200():
    """PyramidNet-272 with alpha=200 for CIFAR-100"""
    return PyramidNet(depth=272, alpha=200, num_classes=100, bottleneck=True)


def pyramidnet200_alpha240():
    """PyramidNet-200 with alpha=240 for CIFAR-100"""
    return PyramidNet(depth=200, alpha=240, num_classes=100, bottleneck=True)

## Training Configuration and Hyperparameters

In [6]:
# Training hyperparameters for state-of-the-art performance
config = {
    'batch_size': 64,  # Smaller batch size for better generalization
    'epochs': 50,      # Reasonable training duration (30-50 range)
    'learning_rate': 0.05,  # Higher initial LR for large batch training
    'momentum': 0.9,
    'weight_decay': 1e-4,
    'nesterov': True,

    # Learning rate schedule
    'lr_schedule': 'cosine',  # Cosine annealing
    'warmup_epochs': 5,
    'min_lr': 1e-6,

    # Label smoothing
    'label_smoothing': 0.1,

    # Model configuration
    'model_depth': 272,
    'model_alpha': 200,
}

print("Training Configuration:")
for key, value in config.items():
    print(f"  {key}: {value}")

Training Configuration:
  batch_size: 64
  epochs: 50
  learning_rate: 0.05
  momentum: 0.9
  weight_decay: 0.0001
  nesterov: True
  lr_schedule: cosine
  warmup_epochs: 5
  min_lr: 1e-06
  label_smoothing: 0.1
  model_depth: 272
  model_alpha: 200


## Training Functions and Utilities

In [7]:
class LabelSmoothingCrossEntropy(nn.Module):
    """Label smoothing cross entropy loss"""
    def __init__(self, smoothing=0.1):
        super().__init__()
        self.smoothing = smoothing

    def forward(self, pred, target):
        log_prob = F.log_softmax(pred, dim=-1)
        weight = pred.new_ones(pred.size()) * self.smoothing / (pred.size(-1) - 1.)
        weight.scatter_(-1, target.unsqueeze(-1), (1. - self.smoothing))
        loss = (-weight * log_prob).sum(dim=-1).mean()
        return loss


def cosine_annealing_lr(epoch, total_epochs, initial_lr, min_lr=1e-6, warmup_epochs=5):
    """Cosine annealing learning rate with warmup"""
    if epoch < warmup_epochs:
        # Linear warmup
        return initial_lr * (epoch + 1) / warmup_epochs
    else:
        # Cosine annealing
        epoch_adjusted = epoch - warmup_epochs
        total_adjusted = total_epochs - warmup_epochs
        return min_lr + (initial_lr - min_lr) * \
               0.5 * (1 + math.cos(math.pi * epoch_adjusted / total_adjusted))


def calculate_accuracy(outputs, targets, topk=(1, 5)):
    """Calculate top-k accuracy"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = targets.size(0)

        _, pred = outputs.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(targets.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res


def train_epoch(model, train_loader, criterion, optimizer, device, epoch):
    """Train for one epoch"""
    model.train()

    running_loss = 0.0
    running_acc1 = 0.0
    running_acc5 = 0.0
    num_samples = 0

    pbar = tqdm(train_loader, desc=f'Epoch {epoch+1} [Train]')

    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        batch_size = data.size(0)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        # Calculate accuracies
        acc1, acc5 = calculate_accuracy(output, target, topk=(1, 5))

        # Update running statistics
        running_loss += loss.item() * batch_size
        running_acc1 += acc1.item() * batch_size
        running_acc5 += acc5.item() * batch_size
        num_samples += batch_size

        # Update progress bar
        pbar.set_postfix({
            'Loss': f'{running_loss/num_samples:.4f}',
            'Acc@1': f'{running_acc1/num_samples:.2f}%',
            'Acc@5': f'{running_acc5/num_samples:.2f}%'
        })

    return running_loss / num_samples, running_acc1 / num_samples, running_acc5 / num_samples


def validate(model, val_loader, criterion, device):
    """Validate the model"""
    model.eval()

    running_loss = 0.0
    running_acc1 = 0.0
    running_acc5 = 0.0
    num_samples = 0

    with torch.no_grad():
        pbar = tqdm(val_loader, desc='[Validation]')

        for data, target in pbar:
            data, target = data.to(device), target.to(device)
            batch_size = data.size(0)

            output = model(data)
            loss = criterion(output, target)

            # Calculate accuracies
            acc1, acc5 = calculate_accuracy(output, target, topk=(1, 5))

            # Update running statistics
            running_loss += loss.item() * batch_size
            running_acc1 += acc1.item() * batch_size
            running_acc5 += acc5.item() * batch_size
            num_samples += batch_size

            # Update progress bar
            pbar.set_postfix({
                'Loss': f'{running_loss/num_samples:.4f}',
                'Acc@1': f'{running_acc1/num_samples:.2f}%',
                'Acc@5': f'{running_acc5/num_samples:.2f}%'
            })

    return running_loss / num_samples, running_acc1 / num_samples, running_acc5 / num_samples

## Model Creation and Training

In [8]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'],
                         shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'],
                        shuffle=False, num_workers=2, pin_memory=True)

# Create model
model = pyramidnet272_alpha200().to(device)

# Count parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Model: PyramidNet-{config['model_depth']} (alpha={config['model_alpha']})")
print(f"Total parameters: {count_parameters(model):,}")
print(f"Model size: {count_parameters(model) / 1e6:.2f}M parameters")

# Loss function and optimizer
criterion = LabelSmoothingCrossEntropy(smoothing=config['label_smoothing'])
optimizer = optim.SGD(model.parameters(),
                     lr=config['learning_rate'],
                     momentum=config['momentum'],
                     weight_decay=config['weight_decay'],
                     nesterov=config['nesterov'])

# Learning rate scheduler
scheduler = optim.lr_scheduler.LambdaLR(
    optimizer,
    lambda epoch: cosine_annealing_lr(
        epoch, config['epochs'], 1.0,
        config['min_lr'] / config['learning_rate'],
        config['warmup_epochs']
    )
)

print("\nTraining setup complete!")
print(f"Training on device: {device}")
print(f"Batch size: {config['batch_size']}")
print(f"Total epochs: {config['epochs']}")
print(f"Learning rate: {config['learning_rate']} (with cosine annealing)")

Model: PyramidNet-272 (alpha=200)
Total parameters: 1,645,971
Model size: 1.65M parameters

Training setup complete!
Training on device: cuda
Batch size: 64
Total epochs: 50
Learning rate: 0.05 (with cosine annealing)


## Main Training Loop

In [9]:
# Training history
history = {
    'train_loss': [], 'train_acc1': [], 'train_acc5': [],
    'test_loss': [], 'test_acc1': [], 'test_acc5': [],
    'lr': []
}

best_acc1 = 0.0
start_time = time.time()

print("Starting training...\n")

for epoch in range(config['epochs']):
    # Update learning rate
    scheduler.step()
    current_lr = optimizer.param_groups[0]['lr']

    # Training
    train_loss, train_acc1, train_acc5 = train_epoch(model, train_loader, criterion, optimizer, device, epoch)

    # Validation (test on CIFAR-100)
    test_loss, test_acc1, test_acc5 = validate(model, test_loader, criterion, device)

    # Save history
    history['train_loss'].append(train_loss)
    history['train_acc1'].append(train_acc1)
    history['train_acc5'].append(train_acc5)
    history['test_loss'].append(test_loss)
    history['test_acc1'].append(test_acc1)
    history['test_acc5'].append(test_acc5)
    history['lr'].append(current_lr)

    # Print epoch summary
    elapsed = time.time() - start_time
    print(f"\nEpoch {epoch+1}/{config['epochs']} Summary:")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc@1: {train_acc1:.2f}% | Train Acc@5: {train_acc5:.2f}%")
    print(f"  Test Loss:  {test_loss:.4f} | Test Acc@1:  {test_acc1:.2f}% | Test Acc@5:  {test_acc5:.2f}%")
    print(f"  Learning Rate: {current_lr:.6f} | Elapsed: {elapsed/3600:.2f}h")

    # Save best model
    if test_acc1 > best_acc1:
        best_acc1 = test_acc1
        torch.save({
            'epoch': epoch,
            'state_dict': model.state_dict(),
            'best_acc1': best_acc1,
            'optimizer': optimizer.state_dict(),
            'config': config
        }, 'best_pyramidnet_cifar100.pth')
        print(f"  🎯 New best accuracy: {best_acc1:.2f}%")

    print("-" * 80)

total_time = time.time() - start_time
print(f"\n✅ Training completed!")
print(f"Total time: {total_time/3600:.2f} hours")
print(f"Best test accuracy: {best_acc1:.2f}%")

/usr/local/lib/python3.12/dist-packages/torch/optim/lr_scheduler.py:192: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn(


Starting training...



[Validation]: 100%|██████████| 157/157 [00:09<00:00, 16.02it/s, Loss=4.4429, Acc@1=5.35%, Acc@5=18.41%]



Epoch 1/50 Summary:
  Train Loss: 4.5614 | Train Acc@1: 2.46% | Train Acc@5: 9.83%
  Test Loss:  4.4429 | Test Acc@1:  5.35% | Test Acc@5:  18.41%
  Learning Rate: 0.020000 | Elapsed: 0.05h
  🎯 New best accuracy: 5.35%
--------------------------------------------------------------------------------


[Validation]: 100%|██████████| 157/157 [00:10<00:00, 15.56it/s, Loss=4.4049, Acc@1=8.75%, Acc@5=26.41%]


...


Epoch 48/50 Summary:
  Train Loss: 1.8736 | Train Acc@1: 67.72% | Train Acc@5: 89.49%
  Test Loss:  1.7539 | Test Acc@1:  71.76% | Test Acc@5:  91.93%
  Learning Rate: 0.000244 | Elapsed: 2.57h
--------------------------------------------------------------------------------


[Validation]: 100%|██████████| 157/157 [00:10<00:00, 15.63it/s, Loss=1.7435, Acc@1=72.03%, Acc@5=92.16%]



Epoch 49/50 Summary:
  Train Loss: 1.8609 | Train Acc@1: 68.19% | Train Acc@5: 89.68%
  Test Loss:  1.7435 | Test Acc@1:  72.03% | Test Acc@5:  92.16%
  Learning Rate: 0.000062 | Elapsed: 2.62h
--------------------------------------------------------------------------------


[Validation]: 100%|██████████| 157/157 [00:10<00:00, 15.32it/s, Loss=1.7473, Acc@1=71.72%, Acc@5=92.10%]


Epoch 50/50 Summary:
  Train Loss: 1.8694 | Train Acc@1: 68.01% | Train Acc@5: 89.53%
  Test Loss:  1.7473 | Test Acc@1:  71.72% | Test Acc@5:  92.10%
  Learning Rate: 0.000001 | Elapsed: 2.68h
--------------------------------------------------------------------------------

✅ Training completed!
Total time: 2.68 hours
Best test accuracy: 72.11%


## Final Evaluation and Results

In [ ]:
# Load best model for final evaluation
checkpoint = torch.load('best_pyramidnet_cifar100.pth')
model.load_state_dict(checkpoint['state_dict'])
best_epoch = checkpoint['epoch']
best_acc = checkpoint['best_acc1']

print(f"Final Evaluation - Best Model (Epoch {best_epoch+1})")
print("=" * 60)

# Final test evaluation
criterion_eval = nn.CrossEntropyLoss()  # No label smoothing for evaluation
final_loss, final_acc1, final_acc5 = validate(model, test_loader, criterion_eval, device)

print(f"\n📊 Final Test Results:")
print(f"  Test Accuracy (Top-1): {final_acc1:.2f}%")
print(f"  Test Accuracy (Top-5): {final_acc5:.2f}%")
print(f"  Test Loss: {final_loss:.4f}")

# Compare with reported results
target_accuracy = 89.3
print(f"\n🎯 Target Accuracy (PyramidNet + ShakeDrop): {target_accuracy}%")
print(f"📈 Achieved Accuracy: {final_acc1:.2f}%")
print(f"📊 Difference: {final_acc1 - target_accuracy:+.2f}%")

if final_acc1 >= target_accuracy:
    print("🏆 SUCCESS: Achieved state-of-the-art performance!")
elif final_acc1 >= target_accuracy - 1.0:
    print("🥈 EXCELLENT: Very close to state-of-the-art!")
elif final_acc1 >= target_accuracy - 2.0:
    print("🥉 GOOD: Strong performance!")
else:
    print("📝 Room for improvement. Consider:")
    print("   - Longer training (300+ epochs)")
    print("   - Better data augmentation")
    print("   - Hyperparameter tuning")
    print("   - Test-time augmentation")

## Training History Visualization

In [ ]:
def plot_training_history(history):
    """Plot comprehensive training history"""
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))

    epochs = range(1, len(history['train_loss']) + 1)

    # Plot loss
    axes[0, 0].plot(epochs, history['train_loss'], 'b-', label='Training Loss', alpha=0.8)
    axes[0, 0].plot(epochs, history['test_loss'], 'r-', label='Test Loss', alpha=0.8)
    axes[0, 0].set_title('Training and Test Loss', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)

    # Plot top-1 accuracy
    axes[0, 1].plot(epochs, history['train_acc1'], 'b-', label='Training Acc@1', alpha=0.8)
    axes[0, 1].plot(epochs, history['test_acc1'], 'r-', label='Test Acc@1', alpha=0.8)
    axes[0, 1].axhline(y=89.3, color='g', linestyle='--', alpha=0.7, label='Target (89.3%)')
    axes[0, 1].set_title('Top-1 Accuracy', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy (%)')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)

    # Plot top-5 accuracy
    axes[1, 0].plot(epochs, history['train_acc5'], 'b-', label='Training Acc@5', alpha=0.8)
    axes[1, 0].plot(epochs, history['test_acc5'], 'r-', label='Test Acc@5', alpha=0.8)
    axes[1, 0].set_title('Top-5 Accuracy', fontsize=14, fontweight='bold')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Accuracy (%)')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

    # Plot learning rate
    axes[1, 1].plot(epochs, history['lr'], 'g-', linewidth=2)
    axes[1, 1].set_title('Learning Rate Schedule', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Learning Rate')
    axes[1, 1].set_yscale('log')
    axes[1, 1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Print key statistics
    print("\n📈 Training Statistics:")
    print(f"  Final Train Accuracy: {history['train_acc1'][-1]:.2f}%")
    print(f"  Final Test Accuracy:  {history['test_acc1'][-1]:.2f}%")
    print(f"  Best Test Accuracy:   {max(history['test_acc1']):.2f}%")
    print(f"  Final Test Top-5:     {history['test_acc5'][-1]:.2f}%")
    print(f"  Best Test Top-5:      {max(history['test_acc5']):.2f}%")

# Plot the training history
if len(history['train_loss']) > 0:
    plot_training_history(history)
else:
    print("No training history to plot. Please run the training first.")

## Model Summary and Architecture Details

In [ ]:
# Print detailed model information
print("🏗️  PyramidNet Architecture Summary")
print("=" * 50)
print(f"Model: PyramidNet-{config['model_depth']}")
print(f"Alpha (channel growth): {config['model_alpha']}")
print(f"Total parameters: {count_parameters(model):,}")
print(f"Model size: {count_parameters(model) / 1e6:.2f}M parameters")

print("\n🔧 Key Features:")
print("  ✅ PyramidNet architecture with gradual channel increase")
print("  ✅ ShakeDrop regularization with linear decay")
print("  ✅ AutoAugment data augmentation policy")
print("  ✅ Label smoothing for better generalization")
print("  ✅ Cosine annealing LR schedule with warmup")
print("  ✅ Random erasing augmentation")
print("  ✅ Bottleneck blocks for efficiency")

print("\n📚 References:")
print("  [1] Deep Pyramidal Residual Networks (https://arxiv.org/abs/1610.02915)")
print("  [2] ShakeDrop regularization (https://arxiv.org/abs/1802.02375)")
print("  [3] AutoAugment (https://arxiv.org/abs/1805.09501)")

print("\n🎯 Target Performance:")
print(f"  CIFAR-100 Top-1 Accuracy: ~89.3%")
print(f"  Achieved in AutoAugment paper with PyramidNet + ShakeDrop")